In [280]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats as stat
import scipy.signal
import math
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from scipy.stats import zscore
from sklearn.metrics import f1_score

## Read data

In [421]:
fileName = '/Users/chenbq/Documents/Find a job/Zest Finance/ZestData.xlsx'
X_train = pd.read_excel(fileName, sheetname = 0)
y_train = pd.read_excel(fileName, sheetname = 1)
X_test = pd.read_excel(fileName, sheetname = 2)
y_test = pd.read_excel(fileName, sheetname = 3)

In [160]:
X_train.columns

Index(['customer_id', 'status', 'residence_rent_or_own', 'monthly_rent_amount',
       'bank_account_direct_deposit', 'application_when', 'loan_duration',
       'payment_ach', 'num_payments', 'address_zip', 'bank_routing_number',
       'home_phone_type', 'monthly_income_amount', 'raw_l2c_score',
       'raw_FICO_telecom', 'raw_FICO_retail', 'raw_FICO_bank_card',
       'raw_FICO_money'],
      dtype='object')

In [405]:
categorical = ['status', 'residence_rent_or_own', 'bank_account_direct_deposit', 'payment_ach', 'home_phone_type']
numerical = ['monthly_rent_amount', 'loan_duration', 'num_payments', 'monthly_income_amount', 'raw_l2c_score', 
             'raw_FICO_telecom', 'raw_FICO_retail', 'raw_FICO_bank_card', 'raw_FICO_money']

In [406]:
def preprocess(df, categorical, numerical):
    df = df.set_index('customer_id')
    
    # Categorical Data
    for cat in categorical:
        tmp = pd.get_dummies(df[cat].astype('category'))
        df = df.merge(tmp, left_index = True, right_index = True, how = 'left')

    # Datetime 
    df['year'] =  [datetime.year for datetime in df['application_when']]
    df['month'] =  [datetime.month for datetime in df['application_when']]
    df['day'] =  [datetime.month for datetime in df['application_when']]
    df['hour'] =  [datetime.month for datetime in df['application_when']]
    
    # Normalize all numerical data
    numerical = numerical + ['year', 'month', 'day', 'hour']
    df[numerical] = df[numerical].apply(zscore)
    
    # Drop columns that are no longer useful
    categorical+=['address_zip', 'bank_routing_number']
    df = df.drop(categorical, axis = 1)
    
    df = df.drop('application_when', axis = 1)
    return df

In [422]:
X = X_train.append(X_test)
X = preprocess(X, categorical, numerical)

In [423]:
X_train = X[:400]
X_test = X[400:]

In [424]:
y_train = y_train.set_index('customer_id')
y_train['value']=[1 if boolean else 0 for boolean in y_train['FullyFunded']] 

In [409]:
len(X_train.columns)

23

## Model: ANN regression

In [425]:
# Save a tmp version
XX_train = X_train
yy_train = y_train

In [450]:
X_train = XX_train
y_train = yy_train
len(X_train)

400

In [451]:
# Train-Dev Split
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.3)
## Tuning parameter alpha
alpha = [0.001]
for i, a in enumerate(alpha): 
    model = MLPClassifier(hidden_layer_sizes=(25, 50, 10  ), activation='logistic', solver='lbfgs', alpha=a)
    model.fit(X_train.values, y_train['value'].values)
    print('alpha=', a, ' Accuracy: ', model.score(X_dev, y_dev['value']))
    print(f1_score(model.predict(X_dev), y_dev['value']))

alpha= 0.001  Accuracy:  0.7416666666666667
0.8287292817679558


In [452]:
model.predict(X_dev)

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [453]:
y_dev['value'].values

array([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [454]:
prediction = model.predict(X_test)

In [457]:
with open('submission.csv', 'w') as f:
    f.write("Id,Predicted\n")
    for i,y in enumerate(prediction):
        f.write("{},{}\n".format(i,bool(y)))